In [1]:
### https://adventofcode.com/2022/day/11
# AoC 2022 - 11_1
import re
import numpy as np

class Monkey:
    def __init__(self, name):
        self.monkey_id = name
        self.items = []
        self.operation = ['+', 0]
        self.divisible = 1
        self.next_monkey = [-1, -1]
        self.inspection_count = 0
  
    def get_values(self, text: str, merge=True):
        return int(''.join(re.findall(r'\b\d+\b', text))) if merge else \
            [int(item) for item in re.findall(r'\b\d+\b', text)]
        
    def parse_monkey(self, _line):
        attr, properties = _line.split(': ')
        if attr == 'Starting items':
            self.items = self.get_values(properties, False)
        if attr == 'Operation':
            if properties.endswith('old'):
                self.operation[0] = '**'
                self.operation[1] = 2
            else:
                self.operation[0] = ''.join([op for op in ['*', '/', '+', '-' ] if op in properties])
                self.operation[1] = self.get_values(properties)
        if attr == 'Test':
            self.divisible = self.get_values(properties)
        if attr == 'If true':
            self.next_monkey[0] = self.get_values(properties)
        if attr == 'If false':
            self.next_monkey[1] = self.get_values(properties)


        
def get_input_data(in_file: str):
    with open(in_file, 'r') as f_in:
        return [_line.strip() for _line in f_in.readlines()]


def monkeys_play(_monkeys: dict, _rounds: int):
    for _round in range(1, _rounds + 1):
        init_op = {
            '+': lambda x, y: x + y,
            '-': lambda x, y: x - y,
            '*': lambda x, y: x * y,
            '/': lambda x, y: x / y,
            '**': lambda x, y: x ** y
        }
        for _id, monkey in _monkeys.items():
            items = monkey.items
            if items:
                for item in items:
                    monkey.inspection_count += 1
                    op = monkey.operation
                    item = init_op[op[0]](item, op[1]) // 3
                    idx = 0 if item % monkey.divisible == 0 else 1
                    next_monkey = _monkeys[str(monkey.next_monkey[idx])]
                    next_monkey.items.append(item)
                monkey.items = []

                
input_data = "aoc2022_data/aoc2022_11.txt"
lines = get_input_data(input_data)

num_rounds = 20
number_most_active = 2
monkeys = dict()

for line in lines:
    if line:
        if line.endswith(':'):
            monkey_name = ''.join(re.findall(r'\b\d+\b', line))
            monkey = Monkey(monkey_name)
        else:
            monkey.parse_monkey(line)
            if 'If false: throw to monkey' in line:
                monkeys[str(monkey.monkey_id)] = monkey

monkeys_play(monkeys, num_rounds)

inspections = []

for monkey in monkeys:
    inspections.append(monkeys[monkey].inspection_count)
    print(f"Monkey {monkeys[monkey].monkey_id}: inpected {monkeys[monkey].inspection_count} times.")

monkey_business = np.prod(sorted(inspections, reverse=True)[:number_most_active])

print(f"Level of monkey business after {num_rounds} rounds is for the {number_most_active} " \
      f"most active monkeys: {monkey_business}.")



Monkey 0: inpected 318 times.
Monkey 1: inpected 296 times.
Monkey 2: inpected 35 times.
Monkey 3: inpected 307 times.
Monkey 4: inpected 39 times.
Monkey 5: inpected 337 times.
Monkey 6: inpected 336 times.
Monkey 7: inpected 40 times.
Level of monkey business after 20 rounds is for the 2 most active monkeys: 113232.


In [2]:
# AoC 2022 - 11_2
import re
import numpy as np
import math

class Monkey:
    def __init__(self, name):
        self.monkey_id = name
        self.items = []
        self.operation = ['+', 0]
        self.divisible = 1
        self.next_monkey = [-1, -1]
        self.inspection_count = 0
  
    def get_values(self, text: str, merge=True):
        return int(''.join(re.findall(r'\b\d+\b', text))) if merge else \
            [int(item) for item in re.findall(r'\b\d+\b', text)]
        
    def parse_monkey(self, _line):
        attr, properties = _line.split(': ')
        if attr == 'Starting items':
            self.items = self.get_values(properties, False)
        if attr == 'Operation':
            if properties.endswith('old'):
                self.operation[0] = '**'
                self.operation[1] = 2
            else:
                self.operation[0] = ''.join([op for op in ['*', '/', '+', '-' ] if op in properties])
                self.operation[1] = self.get_values(properties)
        if attr == 'Test':
            self.divisible = self.get_values(properties)
        if attr == 'If true':
            self.next_monkey[0] = self.get_values(properties)
        if attr == 'If false':
            self.next_monkey[1] = self.get_values(properties)


        
def get_input_data(in_file: str):
    with open(in_file, 'r') as f_in:
        return [_line.strip() for _line in f_in.readlines()]


def monkeys_play(_monkeys: dict, _rounds: int):    
    # see https://de.wikipedia.org/wiki/Prime_Restklassengruppe
    prim_factor = np.prod([monkey.divisible for _id, monkey in _monkeys.items()])
    for _round in range(1, _rounds + 1):
        init_op = {
            '+': lambda x, y: x + y,
            '-': lambda x, y: x - y,
            '*': lambda x, y: x * y,
            '/': lambda x, y: x / y,
            '**': lambda x, y: x ** y
        }
        for _id, monkey in _monkeys.items():
            items = monkey.items
            if items:
                for item in items:
                    monkey.inspection_count += 1
                    op = monkey.operation
                    item = init_op[op[0]](item, op[1])
                    idx = 0 if item % monkey.divisible == 0 else 1
                    next_monkey = _monkeys[str(monkey.next_monkey[idx])]
                    next_monkey.items.append(item % prim_factor)
                monkey.items = []

                
input_data = "aoc2022_data/aoc2022_11.txt"
lines = get_input_data(input_data)

num_rounds = 10000
number_most_active = 2
monkeys = dict()

for line in lines:
    if line:
        if line.endswith(':'):
            monkey_name = ''.join(re.findall(r'\b\d+\b', line))
            monkey = Monkey(monkey_name)
        else:
            monkey.parse_monkey(line)
            if 'If false: throw to monkey' in line:
                monkeys[str(monkey.monkey_id)] = monkey

monkeys_play(monkeys, num_rounds)

inspections = []

for monkey in monkeys:
    inspections.append(monkeys[monkey].inspection_count)
    print(f"Monkey {monkeys[monkey].monkey_id}: inpected {monkeys[monkey].inspection_count} times.")

monkey_business = np.prod(sorted(inspections, reverse=True)[:number_most_active])

print(f"Level of monkey business after {num_rounds} rounds is for the {number_most_active} " \
      f"most active monkeys: {monkey_business}.")

Monkey 0: inpected 171832 times.
Monkey 1: inpected 12942 times.
Monkey 2: inpected 167042 times.
Monkey 3: inpected 12599 times.
Monkey 4: inpected 167043 times.
Monkey 5: inpected 172863 times.
Monkey 6: inpected 12961 times.
Monkey 7: inpected 160272 times.
Level of monkey business after 10000 rounds is for the 2 most active monkeys: 29703395016.


In [3]:
### https://adventofcode.com/2022/day/12
# AoC 2022 - 12_1

# TODO later, NOT FINISHED JET

import itertools
import numpy as np


def get_input_data(in_file: str):
    with open(in_file, 'r') as f_in:
        return [_line.strip() for _line in f_in.readlines()]

    
def get_nodes(_matrix):
    return list(itertools.chain.from_iterable(_matrix))
    
    
def create_graph(_matrix):
    start = (0, 0)
    end = (0, 0)
    num_rows = len(_matrix[:, 0])
    num_cols = len(_matrix[:, 0])
    nodes = []
    print(num_rows, num_cols)
    
    for _row in range(num_rows):
        for _col in range(num_cols):
            if _matrix[_row, _col] == 'S':
                start = (_row, _col)
            if _matrix[_row, _col] == 'E':
                end = (_row, _col)
            else:
                curr_node = _matrix[_row][_col]
                neighbor = None
                # up
                if _row > 0:
                    neighbor = _matrix[_row - 1][_col]
                    if abs(ord(neighbor) - ord(node)) == 1:
                        nodes.append(neighbor)
                # down
                if _row < num_rows - 1:
                    neighbor = _matrix[_row + 1][_col]
                    
                # left
                if _col > 0:
                    neighbor = _matrix[_row][_col - 1]
                    
                # down
                if _col < num_cols - 1:
                    neighbor = _matrix[_row][_col + 1]
                    
                
    
    
input_data = "aoc2022_data/aoc2022_12.txt"
input_data = "aoc2022_data/example.txt"
lines = get_input_data(input_data)
node_matrix = np.array([[*line] for line in lines])
nodes = get_nodes(node_matrix)

print(node_matrix)
print(nodes)
create_graph(node_matrix)





[list(['8', ',', '4', ' ', '-', '>', ' ', '8', ',', '6', ' ', '-', '>', ' ', '6', ',', '6'])
 list(['1', '3', ',', '4', ' ', '-', '>', ' ', '1', '2', ',', '4', ' ', '-', '>', ' ', '1', '2', ',', '9', ' ', '-', '>', ' ', '4', ',', '9'])]
['8', ',', '4', ' ', '-', '>', ' ', '8', ',', '6', ' ', '-', '>', ' ', '6', ',', '6', '1', '3', ',', '4', ' ', '-', '>', ' ', '1', '2', ',', '4', ' ', '-', '>', ' ', '1', '2', ',', '9', ' ', '-', '>', ' ', '4', ',', '9']


<ipython-input-3-91c9ce88b4e4>:59: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  node_matrix = np.array([[*line] for line in lines])


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
# AoC 2022 - 12_2

# TODO later

In [ ]:
### https://adventofcode.com/2022/day/13
# AoC 2022 - 13_1

# TODO later

In [ ]:
# AoC 2022 - 13_2

# TODO later

In [ ]:
### https://adventofcode.com/2022/day/14
# AoC 2022 - 14_1

import numpy as np

        
def get_input_data(in_file: str):
    with open(in_file, 'r') as f_in:
        return [_line.strip() for _line in f_in.readlines()]
    
    
def apply_missing_fields(x_list, y_list):
    x_coord, y_coord = [], []
    for i in range(len(x_list) - 1):
        diff_x = x_list[i] - x_list[i + 1]
        diff_x_abs = abs(diff_x)
        
        diff_y = y_list[i] - y_list[i + 1]
        diff_y_abs = abs(diff_y)
        
        x_coord.append(x_list[i])
        y_coord.append(y_list[i])
        
        if diff_x_abs > 1 and diff_y_abs <= 1:
            for j in range(1, diff_x_abs):
                if diff_x > 1:
                    x_coord.append(x_list[i] - j)
                    y_coord.append(y_list[i])                
                if diff_x < -1:
                    x_coord.append(x_list[i] + j)
                    y_coord.append(y_list[i])  
                                   
        if diff_y_abs > 1 and diff_x_abs <= 1:
            for j in range(1, diff_y_abs):
                if diff_y > 1:
                    y_coord.append(y_list[i] - j)
                    x_coord.append(x_list[i])                
                if diff_y < -1:
                    y_coord.append(y_list[i] + j)
                    x_coord.append(x_list[i]) 
    x_coord.append(x_list[-1]) 
    y_coord.append(y_list[-1]) 
    return x_coord, y_coord


def scan_paths(_lines: str, _abyss_space: int):
    max_x, max_y, _paths = 0, 0, []
    for _line in _lines:
        _line = [int(item) for items in _line.split(' -> ') for item in items.split(',')]
        x_path = _line[::2]
        y_path = _line[1::2]
        x_path, y_path = apply_missing_fields(x_path, y_path)
        max_x, max_y = max(max_x, max(x_path)), max(max_y, max(y_path))
        _paths.append([y_path, x_path])        
    arr = np.chararray((max_y + abyss_space, max_x + abyss_space))
    arr[:] = '.'
    return arr, _paths
    

def draw_paths(_scan,_paths: list):
    for _path in _paths:        
        for row, col in zip(_path[0], _path[1]):
            _scan[row][col] = '#'

            
def sand_runs(scan, start_col):
    count = 0    
    num_rows = len(scan[:,start_col])
    row = 0
    col = start_col
    while True:
        count_start = count
        bottom_blocked = scan[row + 1][col].decode() != '.'
        left_blocked = scan[row + 1][col - 1].decode() != '.'
        right_blocked = scan[row + 1] [col + 1].decode() != '.'
        
        if bottom_blocked:                    
            if left_blocked and not right_blocked:
                col += 1
            elif left_blocked and right_blocked:
                scan[row][col] = 'o'
                count += 1
                row, col = 0, start_col
                continue
            elif not left_blocked:
                col -= 1
                
        row += 1
        if row >= num_rows - 1:
            return scan, count           
           

input_data = "aoc2022_data/aoc2022_14.txt"
lines = get_input_data(input_data)
abyss_space = 10
sand_start_col = 500


ground_scan, paths = scan_paths(lines, abyss_space)
draw_paths(ground_scan, paths)
ground_scan, _count = sand_runs(ground_scan, sand_start_col)

# for line in ground_scan.decode():
#     print(''.join(line))

print(f"{_count} units of sand come to rest before sand starts flowing into the abyss below.")


In [6]:
# AoC 2022 - 14_2

# TODO later, NOT FINISHED JET

import numpy as np

        
def get_input_data(in_file: str):
    with open(in_file, 'r') as f_in:
        return [_line.strip() for _line in f_in.readlines()]
    
    
def apply_missing_fields(x_list, y_list):
    x_coord, y_coord = [], []
    for i in range(len(x_list) - 1):
        diff_x = x_list[i] - x_list[i + 1]
        diff_x_abs = abs(diff_x)
        
        diff_y = y_list[i] - y_list[i + 1]
        diff_y_abs = abs(diff_y)
        
        x_coord.append(x_list[i])
        y_coord.append(y_list[i])
        
        if diff_x_abs > 1 and diff_y_abs <= 1:
            for j in range(1, diff_x_abs):
                if diff_x > 1:
                    x_coord.append(x_list[i] - j)
                    y_coord.append(y_list[i])                
                if diff_x < -1:
                    x_coord.append(x_list[i] + j)
                    y_coord.append(y_list[i])  
                                   
        if diff_y_abs > 1 and diff_x_abs <= 1:
            for j in range(1, diff_y_abs):
                if diff_y > 1:
                    y_coord.append(y_list[i] - j)
                    x_coord.append(x_list[i])                
                if diff_y < -1:
                    y_coord.append(y_list[i] + j)
                    x_coord.append(x_list[i]) 
    x_coord.append(x_list[-1]) 
    y_coord.append(y_list[-1]) 
    return x_coord, y_coord


def scan_paths(_lines: str, space_to_ground: int):
    max_x, max_y, _paths = 0, 0, []
    for _line in _lines:
        _line = [int(item) for items in _line.split(' -> ') for item in items.split(',')]
        x_path = _line[::2]
        y_path = _line[1::2]
        x_path, y_path = apply_missing_fields(x_path, y_path)
        max_x = max(max_x, max(x_path))        
        max_y = max(max_y, max(y_path)) + space_to_ground + 1
        _paths.append([y_path, x_path])        
    estimated_x_range = max_x + 2 * max_y
    arr = np.chararray((max_y, max_x + estimated_x_range))
    arr[:] = '.'
    ground = len(arr[-1,:])
    for i in range(ground):
        arr[-1][i] = '#'        
    return arr, _paths, estimated_x_range // 2 + 1
    

def draw_paths(_scan,_paths: list, left_x):
    for _path in _paths:        
        for row, col in zip(_path[0], _path[1]):
            _scan[row][col + left_x] = '#'

            
def sand_runs(scan, start_col):
    count = 0    
    num_rows = len(scan[:,start_col])
    row = 0
    col = start_col
    while True:
        count_start = count
        bottom_blocked = scan[row + 1][col].decode() != '.'
        left_blocked = scan[row + 1][col - 1].decode() != '.'
        right_blocked = scan[row + 1] [col + 1].decode() != '.'
        
        if bottom_blocked:
#             print(f"bottom blocked: row {row}, col {col}, count: {count}")                      
            if left_blocked and not right_blocked:
                col += 1
            elif left_blocked and right_blocked:
                scan[row][col] = 'o'
                count += 1
                if row == 0:
                    return scan, count  
                row, col = 0, start_col
                continue
            elif not left_blocked:
                col -= 1                
        row += 1
           
        

input_data = "aoc2022_data/aoc2022_14.txt"
input_data = "aoc2022_data/example.txt"
lines = get_input_data(input_data)
sand_start_col = 10
# sand_start_col = 500
space_to_ground = 2


ground_scan, paths, left_x_range = scan_paths(lines, space_to_ground)
sand_start_col += left_x_range
draw_paths(ground_scan, paths, left_x_range)
ground_scan, _count = sand_runs(ground_scan, sand_start_col)

for line in ground_scan.decode():
    print(''.join(line))

print(f"{_count} units of sand come to rest.")


.............................o....................
............................ooo...................
...........................ooooo..................
..........................ooooooo.................
.........................oo#ooo##o................
........................ooo#ooo#ooo...............
.......................oo###ooo#oooo..............
......................oooo.oooo#ooooo.............
.....................oooooooooo#oooooo............
....................ooo#########ooooooo...........
...................ooooo.......ooooooooo..........
##################################################
93 units of sand come to rest.
